In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve
from ast import literal_eval

import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegressionCV, LinearRegression

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, overview, roc_results, model
from lob_data_utils.svm_calculation import lob_svm

sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

In [2]:
data_length = 10000 # TODO: not used
stock = '9061'

In [3]:
df, df_test = lob.load_prepared_data(
    stock, data_dir='../queue_imbalance/data/prepared', cv=False, include_test=True, length=None)
print(len(df), len(df_test))

13796 3449


In [4]:
def prepare_df(df):
    df.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)
    df.drop(columns=['bid', 'ask'], inplace=True)
    df.index = df['datetime']
    df = df.sort_index()
    df['prev_queue_imbalance'] = [None] + df['queue_imbalance'].iloc[0:len(df)-1].values.tolist()
    df.dropna(inplace=True)
    return df
df = prepare_df(df)
df_test = prepare_df(df_test)
df.head()

,datetime,bid_price,ask_price,mid_price,sum_sell_ask,sum_buy_bid,mid_price_indicator,queue_imbalance,prev_queue_imbalance
datetime,,,,,,,,,
2013-09-16 13:07:00,2013-09-16 13:07:00,705.8,706.1,705.95,7224.0,5045.0,0.0,-0.177602,0.007661
2013-09-16 13:08:00,2013-09-16 13:08:00,705.7,705.9,705.80,582.0,2000.0,0.0,0.549187,-0.177602
2013-09-16 13:09:00,2013-09-16 13:09:00,705.6,705.8,705.70,5031.0,1292.0,0.0,-0.591333,0.549187
2013-09-16 13:10:00,2013-09-16 13:10:00,705.3,705.6,705.45,7042.0,5785.0,0.0,-0.097996,-0.591333
2013-09-16 13:11:00,2013-09-16 13:11:00,704.8,705.0,704.90,8155.0,7191.0,1.0,-0.062818,-0.097996


## Let's try to fit Logistic Regression

In [5]:
df_result = pd.DataFrame()
features = ['queue_imbalance', 'prev_queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)

features = ['queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540..."
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540..."


In [6]:
gammas = [0.01, 0.1, 1, 10, 100, 1000]
cs = [0.01, 0.1, 1, 10, 100, 1000]
coef0s = [0.1, 1, 10, 100]


for g in gammas:
    for c in cs:
        for coef in coef0s:
            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance', 'prev_queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)

            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540...",NaN,NaN,NaN
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540...",NaN,NaN,NaN
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,1.0,0.01
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5,

In [7]:
gammas = [0.01, 0.1, 1, 10, 100, 1000]
cs = [0.01, 0.1, 1, 10, 100, 1000]

for g in gammas:
    for c in cs:
        clf = SVC(kernel='rbf', gamma=g, C=c)
        features = ['queue_imbalance', 'prev_queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)

        clf = SVC(kernel='sigmoid', gamma=g, C=c)
        features = ['queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540...",NaN,NaN,NaN
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540...",NaN,NaN,NaN
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,0.1,0.01
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.01,1.0,0.01
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5,

In [8]:
cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

for c in cs:
    clf = SVC(kernel='linear', C=c)
    features = ['queue_imbalance', 'prev_queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)

    clf = SVC(kernel='linear', gamma=g, C=c)
    features = ['queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.42424242424242425, 0.43022035676810066, 0.4...","queue_imbalance, prev_queue_imbalance","[0.041247069635583444, 0.13069866130131835, 0....","[0.04248113431538576, 0.14770649150142107, 0.0...",logistic,"[0.4954337899543379, 0.6193353474320241, 0.572...","[0.37094017094017095, 0.3295819935691318, 0.35...","[0.5202981871143306, 0.5651074524807684, 0.539...","[0.4689655172413793, 0.4029458179905313, 0.464...","[0.12222507928344906, 0.06915130174542727, 0.0...","[0.12642728329008035, 0.07417254874755884, 0.0...","[0.5547785547785548, 0.5246575342465754, 0.530...","[0.4061433447098976, 0.32707087959009395, 0.41...","[0.5601076004986614, 0.5337673021735062, 0.540...",NaN,NaN,NaN
1,"[0.3955375253549696, 0.41431670281995664, 0.39...",queue_imbalance,"[0.02592294278184848, 0.13515915952734903, 0.0...","[0.027181104086745584, 0.1577546054494029, 0.0...",logistic,"[0.486284289276808, 0.6366666666666667, 0.5820...","[0.3333333333333333, 0.3070739549839228, 0.293...","[0.5127055306427504, 0.5673028002767715, 0.536...","[0.42738589211618255, 0.3806451612903226, 0.41...","[0.09614540896270751, 0.05310165499819408, 0.0...","[0.10225901593329777, 0.0578341827812053, 0.08...","[0.544973544973545, 0.5137880986937591, 0.5466...","[0.3515358361774744, 0.3023057216054654, 0.333...","[0.5470254031186774, 0.5258723821191127, 0.540...",NaN,NaN,NaN
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,0.1,0.01
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,0.1,0.01
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]",0.010,1.0,0.01
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",queue_imbalance,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",svm_sigmoid,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5, 0.5, 0

In [9]:
df_result['stock'] = [stock] * len(df_result)
df_result.to_csv('res_{}_prev_queue_imbalance.csv'.format(stock))

In [10]:
df_result.sort_values(by='f1', ascending=False).head(5)

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma,stock
175,"[0.5849802371541502, 0.32858707557502737, 0.62...",queue_imbalance,"[0.027023432131060288, 0.018132630304330877, 0...","[0.032045118732543725, 0.02138565840257375, 0....",svm_sigmoid,"[0.4758842443729904, 0.5154639175257731, 0.529...","[0.7589743589743589, 0.24115755627009647, 0.76...","[0.5140163274692422, 0.5090281452236558, 0.528...","[0.5667963683527886, 0.37343096234309625, 0.56...","[-0.030047858186307508, 0.018069371483923846, ...","[-0.03658693285565347, 0.019308345631705217, -...","[0.4571129707112971, 0.4817813765182186, 0.471...","[0.7457337883959044, 0.304867634500427, 0.7071...","[0.4843938403057366, 0.5088287312367505, 0.493...",10.0,10.0,10.00,9061
77,"[0.5813953488372092, 0.4777183600713013, 0.465...",queue_imbalance,"[0.04931508885635083, 0.06387868349579862, -0....","[0.05533510243124821, 0.0651275247398041, -0.0...",svm_sigmoid,"[0.4846066134549601, 0.536, 0.5054945054945055...","[0.7264957264957265, 0.43086816720257237, 0.43...","[0.5254302249817945, 0.5318897798038179, 0.495...","[0.6006779661016949, 0.45251641137855575, 0.47...","[0.08559168645433812, -0.005034287998687992, 0...","[0.0969928903531578, -0.005039574746590001, 0....","[0.4983127109111361, 0.46409335727109513, 0.48...","[0.7559726962457338, 0.44150298889837747, 0.47...","[0.5441540127935256, 0.49749046228763305, 0.50...",10.0,1.0,0.10,9061
45,"[0.5794520547945206, 0.47636039250669043, 0.46...",queue_imbalance,"[0.04600973869085234, 0.06226926104935415, -0....","[0.0515379469328974, 0.06350731056782379, -0.0...",svm_sigmoid,"[0.48342857142857143, 0.5350701402805611, 0.50...","[0.7230769230769231, 0.4292604501607717, 0.431...","[0.5237208232723928, 0.5310859212829175, 0.494...","[0.6006779661016949, 0.45251641137855575, 0.48...","[0.08559168645433812, -0.005034287998687992, 0...","[0.0969928903531578, -0.005039574746590001, 0....","[0.4983127109111361, 0.46409335727109513, 0.48...","[0.7559726962457338, 0.44150298889837747, 0.47...","[0.5441540127935256, 0.49749046228763305, 0.50...",1000.0,1.0,0.01,9061
109,"[0.5371517027863777, 0.4746666666666666, 0.463...",queue_imbalance,"[0.054334420374840264, 0.055927380009120786, -...","[0.055374755218451484, 0.05696608866753304, -0...",svm_sigmoid,"[0.4908062234794908, 0.5308151093439364, 0.5, ...","[0.5931623931623932, 0.4292604501607717, 0.432...","[0.5275229006170711, 0.5279213643208922, 0.490...","[0.5397553516819572, 0.4586597040905135, 0.478...","[0.04929173047473201, 0.0012797900094431158, 0...","[0.050469542912743644, 0.0012805892318712346, ...","[0.4889196675900277, 0.4676131322094055, 0.486...","[0.6023890784982935, 0.45004269854824935, 0.47...","[0.5249969344587275, 0.5006384023780888, 0.509...",0.1,1.0,1.00,9061
183,"[0.5363703159441586, 0.444649446494465, 0.5781...",queue_imbalance,"[0.009389389239008783, 0.03785329744279953, 0....","[0.009842274354351439, 0.039155172678589464, 0...",svm_sigmoid,"[0.47036082474226804, 0.5216450216450217, 0.53...","[0.6239316239316239, 0.387459807073955, 0.6328...","[0.5047909240734353, 0.5188881313850788, 0.526...","[0.5305232558139535, 0.4471434997750787, 0.517...","[-0.013081148139625576, 0.009374256789329904, ...","[-0.013806770746424328, 0.00941768037867984, -...","[0.4620253164556962, 0.47243346007604564, 0.47...","[0.6228668941979523, 0.42442356959863364, 0.57...","[0.4933196746438864, 0.5046575589204836, 0.494...",100.0,10.0,10.00,9061
